In [40]:
import pandas as pd

In [41]:
df1 = pd.read_parquet('Additional_Data/df_ContentPages_filtered_net.parquet')
df1.head()

,ID_GodotObject_c,ObjectType_c,source_c,sourceCategory_c,Channel_c,Produkt_c,ChannelTopRessort_c,RessortName_c
0,2000017551570,Artikel,AutorIn,Online only,Panorama,Redaktion,Gesellschaft,Religion
1,2000042916576,Artikel,Zeitung,Zeitung,Gesundheit,Redaktion,(Gesundheit),Cure - das kritische Gesundheitsmagazin
2,2000043219975,Artikel,Redaktion,Online only,Web,Redaktion,Innovationen,Innovationen
3,2000044421805,Artikel,AutorIn,Online only,Web,Redaktion,Games,Konsolen
4,2000045434540,Artikel,Zeitung,Zeitung,Karriere,Redaktion,Leitung & Management,Corporate Social Responsibility


In [42]:
df1 = df1[['ID_GodotObject_c','ChannelTopRessort_c']]

In [43]:
df2 = pd.read_parquet('Additional_Data/df_Postings_filtered_net.parquet')
df2.head()

,ID_Posting,ID_Posting_Parent,ID_GodotObject,ID_CommunityIdentity,createdAt,Comment_wo_controlchars_len,Headline_wo_controlchars_len
0,1017441133,NaN,2000050076866,572484,2017-01-01 00:50:39.700,63.0,0.0
1,1017441173,1.017441e+09,2000050076866,521988,2017-01-01 01:01:10.533,54.0,0.0
2,1017441185,1.017441e+09,2000050076866,225492,2017-01-01 01:06:13.480,19.0,0.0
3,1017441195,NaN,2000050076866,565184,2017-01-01 01:08:14.700,89.0,0.0
4,1017441198,1.017441e+09,2000050076866,565184,2017-01-01 01:09:01.690,0.0,33.0


In [44]:
df2 = df2[['ID_CommunityIdentity','ID_GodotObject']]

In [45]:
dfj = df2.join(df1.set_index('ID_GodotObject_c'),on='ID_GodotObject')

In [46]:
df3 = pd.read_parquet('Additional_Data/df_Clicks_net.parquet')
df3.head()

,ID_GodotObject,ID_CommunityIdentity
0,2000050076866,28153
1,2000050076866,215382
2,2000050076866,600903
3,2000050076866,128236
4,2000050076866,235260


In [47]:
dfj = df3.join(dfj.set_index(['ID_GodotObject','ID_CommunityIdentity']),on=['ID_GodotObject','ID_CommunityIdentity'])

In [48]:
dfj.columns = ["BoardID",'UserID','PostedInTopic']

In [49]:
dfj = dfj.join(df1[['ID_GodotObject_c','ChannelTopRessort_c']].set_index('ID_GodotObject_c'),on='BoardID')

In [50]:
dfj.loc[ dfj.PostedInTopic == dfj.ChannelTopRessort_c,'PostedInTopic'] = True

In [51]:
dfj.head()

,BoardID,UserID,PostedInTopic,ChannelTopRessort_c
0,2000050076866,28153,NaN,Europa
1,2000050076866,215382,NaN,Europa
2,2000050076866,600903,NaN,Europa
3,2000050076866,128236,NaN,Europa
4,2000050076866,235260,NaN,Europa


In [77]:
dfj.columns = ['BoardID','UserID','PostedInTopic','BoardTopic']

In [78]:
df_amount_of_posts = dfj.loc[dfj.PostedInTopic == True].groupby('UserID').count()['BoardID']
df_amount_of_posts = df_amount_of_posts.reset_index()
df_amount_of_posts.columns = ['UserID','PostsAmount']

In [117]:
df_amount_of_unique_topics = dfj.loc[dfj.PostedInTopic == True][['UserID','BoardTopic']].groupby('UserID').nunique()
df_amount_of_unique_topics = df_amount_of_unique_topics.reset_index()
df_amount_of_unique_topics.columns = ['UserID','UniquePostsAmount']

In [80]:
topic_counts = dfj.loc[dfj.PostedInTopic == True].groupby(['UserID','BoardTopic']).size().reset_index(name='count')

In [99]:
most_pop_topic_count = topic_counts.loc[topic_counts.groupby('UserID')['count'].idxmax()][['UserID','count']]
#most_pop_topic_count = most_pop_topic_count.reset_index()
most_pop_topic_count.columns = ['UserID','MostPopPostsTopicAmount']

In [102]:
most_pop_topic_count.loc[most_pop_topic_count.UserID == 28153]

,UserID,MostPopPostsTopicAmount
18383,28153,2


In [137]:
dfjj = dfj.join(df_amount_of_posts.set_index('UserID'), on='UserID').join(df_amount_of_unique_topics.set_index('UserID'), on='UserID').join(most_pop_topic_count.set_index('UserID'), on='UserID')

In [138]:
dfjj.head()

,BoardID,UserID,PostedInTopic,BoardTopic,PostsAmount,UniquePostsAmount,MostPopPostsTopicAmount
0,2000050076866,28153,NaN,Europa,4.0,3.0,2.0
1,2000050076866,215382,NaN,Europa,597.0,58.0,183.0
2,2000050076866,600903,NaN,Europa,1959.0,73.0,384.0
3,2000050076866,128236,NaN,Europa,349.0,34.0,102.0
4,2000050076866,235260,NaN,Europa,20.0,9.0,8.0


In [139]:
dfjj = dfjj[['UserID','PostsAmount','UniquePostsAmount','MostPopPostsTopicAmount']]

In [141]:
dfjj.columns = ['UserID','PostsAmount','TopicsPostedIn','MostPopularTopicPostsAmount']

In [147]:
dfjj = dfjj.drop_duplicates()

In [153]:
dfjj = dfjj.fillna(0)

In [156]:
dfjj.to_parquet('Filtered_Data/user_community_inter.parquet',index=False)